# License Plate Detection

In [31]:
# # installing missing YOLO dependencies
# %pip install lapx>=0.5.2
# # installing OCR library
# %pip install easyocr
# %pip install ultralytics
# %pip install pandas

We are importing the following libraries:
* **ast** for parsing the bounding boxes
* **cv2** for video processing
* **easyocr** for OCR
* **glob** for finding files
* **numpy** for array operations
* **pandas** for dataframes
* **string** for string operations
* **ultralytics** for **YOLO** for object detection

In [32]:
import ast
import cv2 as cv
import easyocr
from glob import glob
import numpy as np
import pandas as pd
import string
from ultralytics import YOLO

## License Plate Detection

**YOLOv8** is capable of detecting cars, buses and trucks very easily without additional trainings from the dataset.It is already trained from the COCO dataset.But license number plates seem to be a bit harder. The model often confuses street signs or just basic backgound noise as a car registration plate. 
<br/>
<br/>
To make things more efficient, we are combining both models - a regular COCO trained YOLOv8 and our number plate detector.If the COCO model spots a car, we will then execute the number plate detector to focus its search within the area marked out by the first model's bounding box. That way, we are only seaarching for number plates when there is a car in the picture.

This is a regular COCO trained YOLOv8 model for car detection.<br/>
`coco_model = YOLO('yolov8n.pt')`

This is our custom model trained on the License Plate Dataset.<br/>
`np_model = YOLO('../model/runs/detect/train/weights/best.pt')`

*best.pt* weight is produced by training our model with +21000 annoted images of license plates for 3 epochs.

In [33]:
coco_model = YOLO('yolov8n.pt')
np_model = YOLO('/mnt/D/Projects/anpr/model/runs/detect/train/weights/best.pt')

The input video is read by glob. Glob is a function that returns all the pathnames matching a pattern.

In [34]:
videos = glob('inputs/embossed.mp4')
print(videos)

['inputs/embossed.mp4']


### STEP 1 Implementing the Car Detection

Get the bounding boxes of all vehicles in our video recording with prediction confidence score and object tracking ID

In [35]:
# Here we are reading input videos by ID. IF there are multiple videos in the folder, we can change the ID accordingly.
video = cv.VideoCapture(videos[0])

# ret is a boolean variable that returns True if the frame is read correctly.
ret = True
frame_number = -1
# We can directly detect multiple vehicles in a single frame like car, motorbike, truck using the COCO dataset.
# The COCO dataset has a list of vehicle class IDs.
# Each vehicle class has a unique ID. For example car is 2, motorbike is 3, truck is 5.
# We can searach this information in https://docs.ultralytics.com/datasets/detect/coco/#dataset-yaml
vehicles = [2,3,5]
vehicle_bounding_boxes = []

# read the 10 first frames
while ret:
    frame_number += 1
    ret, frame = video.read()

    if ret and frame_number < 10:

        # use track() to identify vehicles and track them frame by frame
        detections = coco_model.track(frame, persist=True)[0]
        
        # detections.save_crop('./outputs/')
        # This code saves the detected cars in the first 10 frames by the model in the outputs folder.

        # We can print model predictions for debugging.
        # print(results)

        #  (x1, y1, x2, y2, track_id, score, class_id ) represents a bounding box with track ID and confidence score for the object. x1 and y1 are the coordinates of the top left corner of the bounding box. x2 and y2 are the coordinates of the bottom right corner of the bounding box. class_id is the class ID of the object. track_id is the track ID of the object. score is the confidence score of the object.
        for detection in detections.boxes.data.tolist():

            # We can print detection bounding boxes for debugging
            # print(detection)
            x1, y1, x2, y2, track_id, score, class_id = detection
            # Here we are only tracking vehicles so we can just check if the class ID is in the list of vehicles i.e. 2,3,5 and if the confidence score is greater than 0.5 to avoid false positives and false negatives.

            if int(class_id) in vehicles and score > 0.5:
                # We are appending the detected bounding boxes in the list of vehicle bounding boxes
                vehicle_bounding_boxes.append([x1, y1, x2, y2, track_id, score])

# We can print found bounding boxes for debugging
# print(vehicle_bounding_boxes)
video.release()


0: 640x384 1 person, 1 car, 1 truck, 14.8ms
Speed: 3.9ms preprocess, 14.8ms inference, 1.7ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 person, 1 car, 1 truck, 11.9ms
Speed: 2.6ms preprocess, 11.9ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 384)



0: 640x384 1 person, 1 car, 1 truck, 10.9ms
Speed: 2.5ms preprocess, 10.9ms inference, 2.7ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 person, 1 car, 1 truck, 40.4ms
Speed: 7.7ms preprocess, 40.4ms inference, 3.7ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 person, 1 car, 1 truck, 37.0ms
Speed: 8.2ms preprocess, 37.0ms inference, 4.8ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 person, 1 car, 1 truck, 34.2ms
Speed: 8.9ms preprocess, 34.2ms inference, 3.1ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 person, 1 car, 1 truck, 24.3ms
Speed: 9.6ms preprocess, 24.3ms inference, 2.5ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 person, 2 cars, 1 truck, 17.1ms
Speed: 6.1ms preprocess, 17.1ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 person, 2 cars, 1 motorcycle, 1 truck, 10.9ms
Speed: 2.4ms preprocess, 10.9ms inference, 1.5ms postprocess per image at shape (1, 3, 6

This code currently gathers all the bounding boxes for vehicles in the video and stores them in the `vehicle_bounding_boxes` list. Along with the bounding box coordinates, this list also includes the tracking ID assigned to each identified vehicle. The tracking ID remains consistent from frame to frame, serving as a unique identifier. Additionally, the score indicates the model's confidence level that the particular bounding box indeed contains a vehicle, with values ranging from 0 to 1.

### STEP 2 Implementing the License Plate Detection

Use the bounding box for each vehicle and use the number plate detector model to try to find the corresponding plate within in the confinement of those boxes.

In [36]:
# read video by index
video = cv.VideoCapture(videos[0])

ret = True
frame_number = -1
vehicles = [2,3,5]

# read the 10 first frames
while ret:
    frame_number += 1
    ret, frame = video.read()

    if ret and frame_number < 10:
        
        # vehicle detector
        detections = coco_model.track(frame, persist=True)[0]
        for detection in detections.boxes.data.tolist():
            x1, y1, x2, y2, track_id, score, class_id = detection
            if int(class_id) in vehicles and score > 0.5:
                vehicle_bounding_boxes = []
                vehicle_bounding_boxes.append([x1, y1, x2, y2, track_id, score])
                # Loop through vehicle bounding boxes 
                for bbox in vehicle_bounding_boxes:
                    print(bbox)
                    
                    # roi refers to the region of interest where in this case is Vehicles. It crops the image where the vehicle is using the bounding box coordinates.
                    roi = frame[int(y1):int(y2), int(x1):int(x2)]
                    # We can extract the images of the vehicles using the track ID for debugging.
                    # cv.imwrite(str(track_id) + '.jpg', roi)
                    
                    # Here, we are using the license plate detector model to the cropped image of the vehicle obtained from the bounding box.
                    # In this case without searching the whole frame for a license plate, we can only search the images where a vehicle is present.
                    # There is catch with this approch that we can only detect license plates in the images where the vehicle is present.
                    # Even if the license plate is present in the frame but the vechicle is not detected then our model doesnt work.
                    # We can search the whole frame for a license plate but it increases the processing time.
                    # This is a short coming for our project. 
                    license_plates = np_model(roi)[0]

                    # check every bounding box for a license plate
                    for license_plate in license_plates.boxes.data.tolist():

                        # plate_x1 and plate_y1 are the coordinates of the top left corner of the license plate. plate_x2 and plate_y2 are the coordinates of the bottom right corner of the license plate.
                        # plate_score is the confidence score of the license plate.
                        # _ is the class ID of the license plate.
                        plate_x1, plate_y1, plate_x2, plate_y2, plate_score, _ = license_plate

                        # To verify the license plate, we can print the license plate and the track ID.
                        print(license_plate, 'track_id: ' + str(bbox[4]))

                        # plate is the region of interest where in this case is the license plate.
                        # Here we are using the roi of the vehicle and cropping the license plate using the bounding box coordinates generated by our license plate detector model.
                        plate = roi[int(plate_y1):int(plate_y2), int(plate_x1):int(plate_x2)]

                        # We can save the license plate images for debugging.
                        #cv.imwrite(str(track_id) + '.jpg', plate)
                        
video.release()


0: 640x384 1 person, 1 car, 1 motorcycle, 1 truck, 13.5ms
Speed: 3.9ms preprocess, 13.5ms inference, 1.8ms postprocess per image at shape (1, 3, 640, 384)


[0.0, 630.7423706054688, 160.9162139892578, 837.066162109375, 2.0, 0.784398078918457]

0: 640x512 1 License_Plate, 37.1ms
Speed: 7.6ms preprocess, 37.1ms inference, 6.2ms postprocess per image at shape (1, 3, 640, 512)
[13.242539405822754, 125.55684661865234, 76.01828002929688, 151.45028686523438, 0.4229738116264343, 0.0] track_id: 2.0
[313.8712463378906, 713.7379760742188, 437.93634033203125, 893.076904296875, 5.0, 0.5575668215751648]

0: 640x448 (no detections), 40.8ms
Speed: 6.6ms preprocess, 40.8ms inference, 1.6ms postprocess per image at shape (1, 3, 640, 448)

0: 640x384 1 person, 1 car, 1 motorcycle, 1 truck, 44.6ms
Speed: 8.2ms preprocess, 44.6ms inference, 4.3ms postprocess per image at shape (1, 3, 640, 384)
[0.07380685210227966, 631.0355834960938, 154.92303466796875, 835.1675415039062, 2.0, 0.7771762013435364]

0: 640x512 1 License_Plate, 82.8ms
Speed: 10.2ms preprocess, 82.8ms inference, 3.7ms postprocess per image at shape (1, 3, 640, 512)
[18.55599021911621, 126.43299102

### STEP 3 Preprocess License Plates

In [37]:
# read video by index
video = cv.VideoCapture(videos[0])

ret = True
frame_number = -1
vehicles = [2,3,5]

# read the 10 first frames
while ret:
    frame_number += 1
    ret, frame = video.read()

    if ret and frame_number < 100:
        
        # vehicle detector
        detections = coco_model.track(frame, persist=True)[0]
        for detection in detections.boxes.data.tolist():
            x1, y1, x2, y2, track_id, score, class_id = detection
            if int(class_id) in vehicles and score > 0.5:
                vehicle_bounding_boxes = []
                vehicle_bounding_boxes.append([x1, y1, x2, y2, track_id, score])
                for bbox in vehicle_bounding_boxes:
                    print(bbox)
                    roi = frame[int(y1):int(y2), int(x1):int(x2)]
                    
                    # license plate detector for region of interest
                    license_plates = np_model(roi)[0]
                    # process license plate
                    for license_plate in license_plates.boxes.data.tolist():
                        plate_x1, plate_y1, plate_x2, plate_y2, plate_score, _ = license_plate
                        # crop plate from region of interest
                        plate = roi[int(plate_y1):int(plate_y2), int(plate_x1):int(plate_x2)]
                        
                        # Here we are using cvtColor to convert the image to grayscale. cvtColor is a function in OpenCV that converts an image from one color space to another. It is done in this way to reduce the number of colors in an image and increase the probability of detecting the letters in the image using OCR.
                        plate_gray = cv.cvtColor(plate, cv.COLOR_BGR2GRAY)

                        # Here we are using threshold to convert the grayscale image to binary. Posterize is a technique used to reduce the number of colors in an image. Thresholding takes the grayscale image and the pixels that are lower than 64 are set to 255 and the pixels that are higher than 64 are set to 0.
                        #This is inverse beacause we are using THRESH_BINARY_INV in the threshold function.
                        _, plate_treshold = cv.threshold(plate_gray, 64, 255, cv.THRESH_BINARY_INV)
                        
                        # We can print the grayscale images and the threshold images for debugging
                        # cv.imwrite(str(track_id) + '_gray.jpg', plate_gray)
                        # cv.imwrite(str(track_id) + '_thresh.jpg', plate_treshold)
                        
video.release()


0: 640x384 1 person, 1 car, 1 motorcycle, 1 truck, 30.9ms
Speed: 9.4ms preprocess, 30.9ms inference, 3.4ms postprocess per image at shape (1, 3, 640, 384)
[0.0, 630.752685546875, 160.3037109375, 836.9353637695312, 2.0, 0.784398078918457]

0: 640x512 1 License_Plate, 26.4ms
Speed: 5.9ms preprocess, 26.4ms inference, 3.3ms postprocess per image at shape (1, 3, 640, 512)
[317.7317199707031, 712.4478149414062, 444.5315246582031, 894.3303833007812, 5.0, 0.5575668215751648]

0: 640x448 (no detections), 32.0ms
Speed: 5.3ms preprocess, 32.0ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 448)

0: 640x384 1 person, 1 car, 1 motorcycle, 1 truck, 28.6ms
Speed: 7.8ms preprocess, 28.6ms inference, 3.3ms postprocess per image at shape (1, 3, 640, 384)
[0.1554429531097412, 631.0448608398438, 154.55064392089844, 835.08642578125, 2.0, 0.7771762013435364]

0: 640x512 1 License_Plate, 36.5ms
Speed: 3.5ms preprocess, 36.5ms inference, 3.5ms postprocess per image at shape (1, 3, 640, 512)
[3

### STEP 4 Read License Plates

In [38]:
# Initialize the OCR reader
# EasyOCR is an open-source library for optical character recognition (OCR) that is used for text recognition.
# 'en' is the language of the OCR reader.
reader = easyocr.Reader(['en'], gpu=True)

In [39]:
#Here we are making a function named read_license_plate that takes in the license plate crop and returns the license plate number and the license plate score using the OCR reader. reader variable is an instance of the EasyOCR reader.
# reader.readtext is a function in the EasyOCR library that takes in the license plate crop and returns the license plate number and the license plate score.
def read_license_plate(license_plate_crop):
    detections = reader.readtext(license_plate_crop)

    for detection in detections:
        # It gives the bounding box, the text, and the confidence score of the license plate.
        bbox, text, score = detection

        # We are using the upper function to convert the License Plate text to uppercase.
        text = text.upper().replace(' ', '')
        
        return text, score

    return None, None

In [40]:
# write_csv is a function that writes the obtained results to a CSV file using the specified format.
# Here we are formatting the colunms as [frame_number, track_id, car_bbox, car_bbox_score, license_plate_bbox, license_plate_bbox_score, license_plate_number, license_text_score].
# car_bbox and license_plate_bbox has 4 array that stores the coordinate of the bounding box.

def write_csv(results, output_path):
    
    with open(output_path, 'w') as f:
        f.write('{},{},{},{},{},{},{},{}\n'.format(
            'frame_number', 'track_id', 'car_bbox', 'car_bbox_score',
            'license_plate_bbox', 'license_plate_bbox_score', 'license_plate_number',
            'license_text_score'))

        for frame_number in results.keys():
            for track_id in results[frame_number].keys():
                print(results[frame_number][track_id])
                if 'car' in results[frame_number][track_id].keys() and \
                   'license_plate' in results[frame_number][track_id].keys() and \
                   'number' in results[frame_number][track_id]['license_plate'].keys():
                    f.write('{},{},{},{},{},{},{},{}\n'.format(
                        frame_number,
                        track_id,
                        '[{} {} {} {}]'.format(
                            results[frame_number][track_id]['car']['bbox'][0],
                            results[frame_number][track_id]['car']['bbox'][1],
                            results[frame_number][track_id]['car']['bbox'][2],
                            results[frame_number][track_id]['car']['bbox'][3]
                        ),
                        results[frame_number][track_id]['car']['bbox_score'],
                        '[{} {} {} {}]'.format(
                            results[frame_number][track_id]['license_plate']['bbox'][0],
                            results[frame_number][track_id]['license_plate']['bbox'][1],
                            results[frame_number][track_id]['license_plate']['bbox'][2],
                            results[frame_number][track_id]['license_plate']['bbox'][3]
                        ),
                        results[frame_number][track_id]['license_plate']['bbox_score'],
                        results[frame_number][track_id]['license_plate']['number'],
                        results[frame_number][track_id]['license_plate']['text_score'])
                    )
        f.close()

### STEP 5 Clean-Up License Plate Format

In [41]:
# Here we are mapping dictionaries for character conversion.
# If we know that the first character in the number plate always is an string eg. `O` then if our OCR reader reads that O as `0` then it would be a mistake.
# To prevent this we are mapping dictionaries with similar keys and values.

dict_char_to_int = {'O': '0',
                    'I': '1',
                    'J': '3',
                    'A': '4',
                    'G': '6',
                    'S': '5'}

dict_int_to_char = {'0': 'O',
                    '1': 'I',
                    '3': 'J',
                    '4': 'A',
                    '6': 'G',
                    '5': 'S'}

In [42]:
# license_complies_format is a function that checks if the license plate complies with the specified format.
# In this case the format is `[A-Z][A-Z][0-9][0-9][A-Z][A-Z][A-Z]`.
# We can change this format for specific use cases. For example, now it is configured for UK number plates. We can change the format according to Nepali number plates for our use.
# The above character conversion comes handy in this situation where if we are sure that in the second letter of our text we should get a string then if our OCR Reader reads a integer that looks similar to a alphabet maybe 4 then we can neglect the '4 and read 'A' instead.
def license_complies_format(text):
    # It returnsTrue if the license plate complies with the format, False otherwise.
    if len(text) != 7:
        return False

    if (text[0] in string.ascii_uppercase or text[0] in dict_int_to_char.keys()) and \
       (text[1] in string.ascii_uppercase or text[1] in dict_int_to_char.keys()) and \
       (text[2] in ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'] or text[2] in dict_char_to_int.keys()) and \
       (text[3] in ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'] or text[3] in dict_char_to_int.keys()) and \
       (text[4] in string.ascii_uppercase or text[4] in dict_int_to_char.keys()) and \
       (text[5] in string.ascii_uppercase or text[5] in dict_int_to_char.keys()) and \
       (text[6] in string.ascii_uppercase or text[6] in dict_int_to_char.keys()):
        return True
    else:
        return False

In [43]:

def format_license(text):
    license_plate_ = ''
    mapping = {0: dict_int_to_char, 1: dict_int_to_char, 4: dict_int_to_char, 5: dict_int_to_char, 6: dict_int_to_char,
               2: dict_char_to_int, 3: dict_char_to_int}
    for j in [0, 1, 2, 3, 4, 5, 6]:
        if text[j] in mapping[j].keys():
            license_plate_ += mapping[j][text[j]]
        else:
            license_plate_ += text[j]

    return license_plate_

This returns a list with bounding box metrics for every frame with a successful detection.

In [44]:
def read_license_plate(license_plate_crop):
    detections = reader.readtext(license_plate_crop)

    for detection in detections:
        bbox, text, score = detection

        text = text.upper().replace(' ', '')

        # verify that text is confirmed to a standard license plate
        if license_complies_format(text):
            # bring text into the default license plate format
            return format_license(text), score

    return None, None

In [45]:
results = {}

# read video by index
video = cv.VideoCapture(videos[0])

ret = True
frame_number = -1
vehicles = [2,3,5]

# read the entire video
while ret:
    ret, frame = video.read()
    frame_number += 1
    if ret:
        results[frame_number] = {}
        
        # vehicle detector
        detections = coco_model.track(frame, persist=True)[0]
        for detection in detections.boxes.data.tolist():
            x1, y1, x2, y2, track_id, score, class_id = detection
            if int(class_id) in vehicles and score > 0.5:
                vehicle_bounding_boxes = []
                vehicle_bounding_boxes.append([x1, y1, x2, y2, track_id, score])
                for bbox in vehicle_bounding_boxes:
                    print(bbox)
                    roi = frame[int(y1):int(y2), int(x1):int(x2)]
                    
                    # license plate detector for region of interest
                    license_plates = np_model(roi)[0]
                    # process license plate
                    for license_plate in license_plates.boxes.data.tolist():
                        plate_x1, plate_y1, plate_x2, plate_y2, plate_score, _ = license_plate
                        # crop plate from region of interest
                        plate = roi[int(plate_y1):int(plate_y2), int(plate_x1):int(plate_x2)]
                        # cv.imwrite('outputs/plates/roi/'+str(track_id)+ '.jpg', plate)
                        # de-colorize
                        plate_gray = cv.cvtColor(plate, cv.COLOR_BGR2GRAY)
                        # cv.imwrite('outputs/plates/gray/'+str(track_id)+ '.jpg', plate_gray)
                        # posterize
                        _, plate_treshold = cv.threshold(plate_gray, 64, 255, cv.THRESH_BINARY_INV)
                        # cv.imwrite('outputs/plates/thresh/'+str(track_id)+ '.jpg', plate_treshold)
                        # OCR
                        np_text, np_score = read_license_plate(plate_treshold)
                        # if plate could be read write results
                        if np_text is not None:
                            results[frame_number][track_id] = {
                                'car': {
                                    'bbox': [x1, y1, x2, y2],
                                    'bbox_score': score
                                },
                                'license_plate': {
                                    'bbox': [plate_x1, plate_y1, plate_x2, plate_y2],
                                    'bbox_score': plate_score,
                                    'number': np_text,
                                    'text_score': np_score
                                }
                            }

write_csv(results, './outputs/resultsEmbossed.csv')
video.release()

0: 640x384 5 persons, 1 car, 1 motorcycle, 1 truck, 14.8ms
Speed: 3.9ms preprocess, 14.8ms inference, 1.8ms postprocess per image at shape (1, 3, 640, 384)


ValueError: not enough values to unpack (expected 7, got 6)

In [ ]:
results = pd.read_csv('./outputs/resultsEmbossed.csv')

# show results for tracking ID `1` - sort by OCR prediction confidence
results[results['track_id'] == 1.].sort_values(by='license_text_score', ascending=False)

,frame_number,track_id,car_bbox,car_bbox_score,license_plate_bbox,license_plate_bbox_score,license_plate_number,license_text_score


### STEP 6 Visualize the Results

In [ ]:
def draw_border(img, top_left, bottom_right, color=(0, 255, 0), thickness=6, line_length_x=200, line_length_y=200):
    x1, y1 = top_left
    x2, y2 = bottom_right

    cv.line(img, (x1, y1), (x1, y1 + line_length_y), color, thickness)  #-- top-left
    cv.line(img, (x1, y1), (x1 + line_length_x, y1), color, thickness)

    cv.line(img, (x1, y2), (x1, y2 - line_length_y), color, thickness)  #-- bottom-left
    cv.line(img, (x1, y2), (x1 + line_length_x, y2), color, thickness)

    cv.line(img, (x2, y1), (x2 - line_length_x, y1), color, thickness)  #-- top-right
    cv.line(img, (x2, y1), (x2, y1 + line_length_y), color, thickness)

    cv.line(img, (x2, y2), (x2, y2 - line_length_y), color, thickness)  #-- bottom-right
    cv.line(img, (x2, y2), (x2 - line_length_x, y2), color, thickness)

    return img

In [ ]:
# read video by index
video = cv.VideoCapture(videos[0])

# get video dims
frame_width = int(video.get(3))
frame_height = int(video.get(4))
size = (frame_width, frame_height)

# Define the codec and create VideoWriter object
fourcc = cv.VideoWriter_fourcc(*'mp4v')
out = cv.VideoWriter('./outputs/resultsEmbossed.mp4', fourcc, 20.0, size)

# reset video before you re-run cell below
frame_number = -1
video.set(cv.CAP_PROP_POS_FRAMES, 0)

True

In [ ]:
ret = True

while ret:
    ret, frame = video.read()
    frame_number += 1
    if ret:
        df_ = results[results['frame_number'] == frame_number]
        for index in range(len(df_)):
            # draw car
            vhcl_x1, vhcl_y1, vhcl_x2, vhcl_y2 = ast.literal_eval(df_.iloc[index]['car_bbox'].replace('[ ', '[').replace('   ', ' ').replace('  ', ' ').replace(' ', ','))
            
            draw_border(
                frame, (int(vhcl_x1), int(vhcl_y1)),
                (int(vhcl_x2), int(vhcl_y2)), (0, 255, 0),
                12, line_length_x=200, line_length_y=200)
            
            # draw license plate
            plate_x1, plate_y1, plate_x2, plate_y2 = ast.literal_eval(df_.iloc[index]['license_plate_bbox'].replace('[ ', '[').replace('   ', ' ').replace('  ', ' ').replace(' ', ','))

            # region of interest for license plate
            roi = frame[int(vhcl_y1):int(vhcl_y2), int(vhcl_x1):int(vhcl_x2)]
            cv.rectangle(roi, (int(plate_x1), int(plate_y1)), (int(plate_x2), int(plate_y2)), (0, 0, 255), 6)
            #endregion
            # write detected number
            (text_width, text_height), _ = cv.getTextSize(
                df_.iloc[index]['license_plate_number'],
                cv.FONT_HERSHEY_SIMPLEX,
                2,
                6)

            cv.putText(
                frame,
                df_.iloc[index]['license_plate_number'],
                (int((vhcl_x2 + vhcl_x1 - text_width)/2), int(vhcl_y1 - text_height)),
                cv.FONT_HERSHEY_SIMPLEX,
                2,
                (0, 255, 0),
                6
            )

        out.write(frame)
        frame = cv.resize(frame, (1280, 720))

out.release()
video.release()

In [ ]:
import pandas as pd
import numpy as np

# Assuming your input data is stored in a CSV file named 'data.csv'
# You can adjust the file name or provide the data directly if it's not in a file
data = pd.read_csv('./outputs/resultsEmbossed.csv')

# Convert 'license_text_score' to numeric
data['license_text_score'] = pd.to_numeric(data['license_text_score'], errors='coerce')

# Calculate the total sum of license_text_score for each license_plate_number
total_license_score = data.groupby('license_plate_number')['license_text_score'].sum()

# Find the row with the maximum license_plate_score for each license_plate_number
max_license_score_row = data.loc[data.groupby('license_plate_number')['license_text_score'].idxmax()]

# Merge the two DataFrames on license_plate_number
result = pd.merge(max_license_score_row[['license_plate_number', 'track_id']], total_license_score.reset_index(),
                  on='license_plate_number', how='inner')

# Find the row with the maximum license_text_score for each track_id
max_license_score_row = result.loc[result.groupby('track_id')['license_text_score'].idxmax()]

# Display the result
print(max_license_score_row)


  license_plate_number  track_id  license_text_score
2              EA26GOG      10.0            0.697498
